# Transfert Learning with TensorFlow Part1: Feature extraction

TF is leveraging an existing model architecture and learn patterns for our problem:

- Can leverage an existing nerual network proven to work on a similar problem to our own
- Can leverage a working NN so we can adapt the patterns to our own data
- train on a smaller amount of data

## 1. Load and prepare the data

### Load the data

In [1]:
# are we using a GPU ?
!nvidia-smi

Mon Oct  7 16:26:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   46C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
! pip install tensorflow==2.14.0
import tensorflow as tf
print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: google-auth-oauthlib
    Found existing in

In [3]:
# downloading the data
from zipfile import ZipFile

!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"
zip_ref = ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2024-10-07 16:27:34--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  19.2MB/s    in 9.5s    

2024-10-07 16:27:45 (16.9 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:

# inspect our data !
from pathlib import Path

data_dir = Path("10_food_classes_10_percent")
for dirpath in data_dir.rglob('*'):
    if dirpath.is_dir():
        subdirs = [d for d in dirpath.iterdir() if d.is_dir()]
        files = [f for f in dirpath.iterdir() if f.is_file()]
        print(f"There are {len(subdirs)} directories and {len(files)} images in '{dirpath}'.")


There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_

### Creating data loaders (preparing the data)

In [5]:
# setup data inputs
# since we have a nicely formated data folders we use the ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32
EPOCHS = 5

train_dir = data_dir / "train"
test_dir = data_dir / "test"

train_data_gen = ImageDataGenerator(rescale=1/255.)
test_data_gen = ImageDataGenerator(rescale=1/255.)

train_data_10_percent = train_data_gen.flow_from_directory(train_dir,
                                                           target_size=IMAGE_SHAPE,
                                                           batch_size=BATCH_SIZE,
                                                           class_mode="categorical")

test_data_10_percent = test_data_gen.flow_from_directory(test_dir,
                                                         target_size=IMAGE_SHAPE,
                                                         batch_size=BATCH_SIZE,
                                                         class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


### Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality we can add to our models to be performed during or after training. Some of the most popular callbacks are:

- Tracking experiments with the TensorBoard Callback
- Model checkpoint with the ModelCheckpoint callback
- stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback

In [6]:
# let's create a function to build a TensorBoard callback
from datetime import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

### Creating models using TensorFlow Hub

In the past we'have used TensorFlow to create our own models layer by layer from scratch

Now we're going to do a similar process, except the majority of our model's layer will come from TensorFlow Hub

https://www.tensorflow.org/hub?hl=fr

Browsing the TensorFlow Hub page and sorting for image classification, we found the following feature vector model link:
https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1



In [7]:
# let's compare the following two models
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [8]:
# import dependencies
import tensorflow_hub as hub
import tensorflow as tf

In [9]:
print(tf.__version__)
print(hub.__version__)

2.14.0
0.16.1


In [10]:
!pip show tensorflow-hub

Name: tensorflow-hub
Version: 0.16.1
Summary: TensorFlow Hub is a library to foster the publication, discovery, and consumption of reusable parts of machine learning models.
Home-page: https://github.com/tensorflow/hub
Author: Google LLC
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, protobuf, tf-keras
Required-by: 


In [ ]:
#! pip install --upgrade keras

In [11]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
print(hub.__version__)

2.14.0
0.16.1


In [12]:
import keras
print(keras.__version__)

2.14.0


In [13]:
# Let's make a create_model() function to create a model from URL
def create_model(model_url, num_classes=10):
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it
    Args:
      model_url (str): A TensorFlow Hub feature extraction URL
      num_classes (int): Number of output neurons in the output layer,
        should be equal to number of target classes, default 10
      Returns:
      An uncompiled Keras Sequential model with model_url as feature
      extractor layer and Dense outpu layer with num_classes outputs
    """

    print(f"Building model with {model_url}")
    feature_extraction_layer = hub.KerasLayer(model_url,trainable=False)

    model=tf.keras.Sequential([
        tf.keras.Input(shape=(224,224,3)),
        feature_extraction_layer,
        tf.keras.layers.Dense(num_classes,activation='softmax')
    ])
    return model


### Creating and testing ResNet TensorFlow Hub

In [14]:
eff_net = create_model(efficientnet_url)

Building model with https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1


In [15]:
res_net = create_model(resnet_url)

Building model with https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4


### Compiling the models

In [16]:
# efficient net
eff_net.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# resnet
res_net.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


### Evaluate the model

In [17]:
# efficient net
hisotry_eff_net = eff_net.fit(train_data_10_percent, epochs=5, validation_data=test_data_10_percent)

Epoch 1/5
24/24 [==============================] - 30s 920ms/step - loss: 1.8581 - accuracy: 0.4240 - val_loss: 1.2778 - val_accuracy: 0.7248
Epoch 2/5
24/24 [==============================] - 20s 864ms/step - loss: 1.0714 - accuracy: 0.7720 - val_loss: 0.8625 - val_accuracy: 0.8192
Epoch 3/5
24/24 [==============================] - 20s 874ms/step - loss: 0.7671 - accuracy: 0.8373 - val_loss: 0.6982 - val_accuracy: 0.8392
Epoch 4/5
24/24 [==============================] - 20s 862ms/step - loss: 0.6184 - accuracy: 0.8680 - val_loss: 0.6093 - val_accuracy: 0.8540
Epoch 5/5
24/24 [==============================] - 20s 869ms/step - loss: 0.5215 - accuracy: 0.8813 - val_loss: 0.5590 - val_accuracy: 0.8588


In [18]:
history_res_net = res_net.fit(train_data_10_percent, epochs=5, validation_data=test_data_10_percent,
                              callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", experiment_name="resenet50V2")]
                              )

Saving TensorBoard log files to: tensorflow_hub/resenet50V2/20241007-164614
Epoch 1/5
24/24 [==============================] - 51s 2s/step - loss: 1.9762 - accuracy: 0.3627 - val_loss: 1.2111 - val_accuracy: 0.6188
Epoch 2/5
24/24 [==============================] - 50s 2s/step - loss: 0.9181 - accuracy: 0.7347 - val_loss: 0.8438 - val_accuracy: 0.7344
Epoch 3/5
24/24 [==============================] - 47s 2s/step - loss: 0.6207 - accuracy: 0.8387 - val_loss: 0.7531 - val_accuracy: 0.7640
Epoch 4/5
24/24 [==============================] - 47s 2s/step - loss: 0.4844 - accuracy: 0.8760 - val_loss: 0.7166 - val_accuracy: 0.7632
Epoch 5/5
24/24 [==============================] - 47s 2s/step - loss: 0.3822 - accuracy: 0.9080 - val_loss: 0.6887 - val_accuracy: 0.7736


In [19]:
# Let's create a function to plot our curves ....
# we could put a function like this into a script called helper.py and import it when we need it !
from matplotlib import pyplot as plt

def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics
    Args:
      history: TensorFlow model History object
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs=range(len(history.history["loss"]))

    #plot loss
    plt.plot(epochs, loss, label="training_loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("loss")
    plt.xalbel("Epochs")
    plt.legend()

    #plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label="training_accuracy")
    plt.plot(epochs, val_accuracy, label="val_accuracy")
    plt.title("accuracy")
    plt.xlabel("Epochs")
    plt.legend()

## Comparing our models

it is really tedious to compare model by scrolling up each time so the idea is to put callback (tensor board) when runing the models

In [20]:
# upload tensorboard

!tensorboard dev upload --logdir ./tensorflow_hub/ \
--name "Resnet" \
--description "Plotting our experiement" \
--one_shot

2024-10-07 16:52:48.863624: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 16:52:48.863673: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 16:52:48.863705: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 16:52:49.878837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-07 16:52:51.244367: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 